In [1]:
import numpy as np
import os
import pickle

from tqdm import tqdm_notebook
import pandas as pd


In [2]:
mutation_labels = ["Y29F", "Y29A",  "W38F", "T42A", "W44F", "E76A", "E80A", 
                   "D35A", "D39A", "K27A", "R59A", "R83Q", "R87A", "H102A", 
                   "F29Y", "A29Y",  "F38W", "A42T", "F44W", "A76E", "A80E", 
                   "A35D", "A39D", "A27K", "A59R", "Q83R", "A87R", "A102H"]
categories = ["backbone torsions", "sidechain torsions", "intra interface contacts", "inter interface contacts", "neighboring waters"]
phase = 'complex'

data = []
for sub_dir in tqdm_notebook(range(28)):
    print(f"sub_dir: {sub_dir}")
    
    if sub_dir == 4:
        main_dir = 45
        sub_dir_final = 11
        replicate = 0

    elif sub_dir == 12:
        main_dir = 45
        sub_dir_final = 10
        replicate = 1

    elif sub_dir == 17:
        main_dir = 45
        sub_dir_final = 9
        replicate = 1
    
    else:
        main_dir = 47
        replicate = 1
        sub_dir_final = sub_dir

    # Load correlation data -- dict of key: name of degree of freedom, value: dict containing 'pearsonr' and 'tranform_type' as keys
    out_dir = f"/data/chodera/zhangi/perses_benchmark/repex/perses-bnbs-paper-fourth-attempt/{main_dir}/{sub_dir_final}/replicate_{replicate}/"
    with open(os.path.join(out_dir, f"correlation_data_replicas_all_10ns.pickle"), "rb") as f:
        correlation_data = pickle.load(f)
    correlation_data_keys = list(correlation_data.keys())
    print(correlation_data.keys())
    
    # Load pccs and cis 
    with open(os.path.join(out_dir, f"correlation_data_replicas_all_with_cis_10ns.pickle"), "rb") as f:
        correlation_data_cis = pickle.load(f) # First 5 elements are PCCs for each of the dof categories, latter 5 elements are the corresponding CIs
    print(correlation_data_cis)
    
    for i, category in enumerate(categories):
        if i == 4:
            dof_name = f"nearby waters {correlation_data_keys[i]}"
        else:
            dof_name = correlation_data_keys[i]
        data.append([mutation_labels[sub_dir], category, round(correlation_data_cis[i], 2), tuple(round(val, 2) for val in correlation_data_cis[i+5]), dof_name])

/scratch/lsftmp/18362115.tmpdir/ipykernel_51440/3984796847.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for sub_dir in tqdm_notebook(range(28)):


  0%|          | 0/28 [00:00<?, ?it/s]

sub_dir: 0
odict_keys(['barnase-101 psi old', 'barstar-29 chi1 old', 'barstar-29 | barstar-39 old', 'barnase-87 | barstar-29 old', 'old'])
[-0.054791415011235, -0.2500316539060465, 0.19318017133175433, 0.2160416756811001, 0.25763677447909983, (-0.08745427906418965, -0.0267247408546214), (-0.33915459359576966, -0.12624460612451022), (0.12094933637885628, 0.27510429411520665), (0.11094374494268776, 0.29242766122417685), (0.14763262270104005, 0.3511726613445848)]
sub_dir: 1
odict_keys(['barstar-29 psi old', 'barstar-29 chi1 old', 'barstar-27 | barstar-29 old', 'barnase-102 | barstar-29 old', 'old'])
[0.0864291927058418, 0.13035822235010971, 0.0880855709789542, 0.18443679910076397, 0.15618208187515614, (0.05209534119777914, 0.12302882370144859), (0.05078583072264136, 0.20360233279026418), (0.0012005782160306002, 0.17120408370922924), (0.12646432602295116, 0.24014871976979937), (0.09553279474985066, 0.20697433696930922)]
sub_dir: 2
odict_keys(['barstar-38 phi old', 'barstar-38 chi1 old', 'b

In [3]:
df = pd.DataFrame(data, columns=["Mutation", "Degree of freedom category", "PCC", "95% confidence interval", "Degree of freedom"])

In [4]:
df

,Mutation,Degree of freedom category,PCC,95% confidence interval,Degree of freedom
0,Y29F,backbone torsions,-0.05,"(-0.09, -0.03)",barnase-101 psi old
1,Y29F,sidechain torsions,-0.25,"(-0.34, -0.13)",barstar-29 chi1 old
2,Y29F,intra interface contacts,0.19,"(0.12, 0.28)",barstar-29 | barstar-39 old
3,Y29F,inter interface contacts,0.22,"(0.11, 0.29)",barnase-87 | barstar-29 old
4,Y29F,neighboring waters,0.26,"(0.15, 0.35)",nearby waters old
...,...,...,...,...,...
135,A102H,backbone torsions,0.09,"(0.05, 0.13)",barstar-31 psi old
136,A102H,sidechain torsions,-0.06,"(-0.09, -0.03)",barnase-85 chi1 old
137,A102H,intra interface contacts,-0.15,"(-0.19, -0.11)",barstar-29 | barstar-36 old
138,A102H,inter interface contacts,-0.14,"(-0.17, -0.11)",barnase-58 | barstar-29 old


In [5]:
save_path = "/data/chodera/zhangi/perses_benchmark/repex/perses-bnbs-paper-fourth-attempt/47/table_10ns_arex_pccs.csv"
df.to_csv(save_path, index=False)